In [52]:
import pandas as pd
from sqlalchemy import create_engine
import json
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")
import plotly.express as px
import numpy as np


In [2]:
# Read the MySQL configuration from the JSON file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

In [3]:
# Extract MySQL connection details
mysql_config = config.get('mysql', {})
username = mysql_config.get('username', 'default_username')
password = mysql_config.get('password', 'default_password')
host = mysql_config.get('host', 'localhost')
database_name = mysql_config.get('database_name', 'your_database')
table_name = 'measurments_rewa'

In [4]:
# Create the MySQL database connection string
db_url = f"mysql+mysqlconnector://{username}:{password}@{host}/{database_name}"

# Create an SQLAlchemy engine
engine = create_engine(db_url)

# Use the engine to connect to the database
connection = engine.connect()

In [5]:
# Specify the SQL query to retrieve data from a table
query = "SELECT * FROM measurments_rewa"

# Use Pandas to read data from the database into a DataFrame
df = pd.read_sql(query, connection)

# Close the database connection
connection.close()

In [38]:
# Define wind direction bins
bins = [0, 45, 90, 135, 180, 225, 270, 315, 360]
labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

# Bin wind directions and calculate mean wind speed for each bin
df['WindDirBin'] = pd.cut(df['WindDir'], bins=bins, labels=labels)
#aggregated_data = df.groupby('WindDirBin')['WindSpeed'].count().reset_index()
aggregated_data = df.groupby('WindDirBin').agg({'WindSpeed':'mean','WindDir':'count'}).reset_index()

In [55]:
aggregated_data.rename({'WindDir':'Count'}, inplace=True)
aggregated_data['Frequency'] = np.round(aggregated_data['WindDir']/aggregated_data['WindDir'].sum(),2)*100

In [56]:
# Create a wind rose plot
fig = px.bar_polar(aggregated_data, r='Frequency', theta='WindDirBin', color='WindSpeed',
                   template='plotly_dark', color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.show()